# All of Statistics (in high dimensions)
## Chapter 1 - Probability

Re-reading Wasserman's All of Statistics! The idea for this re-read is to focus on high dimensions, from page 1. 

Why? Mostly because it seems all of the statistics applications I work on are in high dimensional settings. And probability itself, in high dimensions, doesn't believe as we think it does. Typically, maximum likelihood is horrifying in dimensions larger than four (as we will see later); in reasonably standard high dimensional sets (like the n-ball), there is way more probability mass at the limit of the set than everywhere else, and so on.

The idea is to clearly show these as we go along and hopefully pick up on new high dimensional intuitions.

Most of these ideas also stem, directly or indirectly, from [Introduction to High Dimensional Statistics](https://www.crcpress.com/Introduction-to-High-Dimensional-Statistics/Giraud/p/book/9781482237948), by Christophe Giraud, and by [High-Dimensional Data Analysis: The Curses and Blessings of Dimensionality](http://statweb.stanford.edu/~donoho/Lectures/AMS2000/AMS2000.html) by David Donoho.

### Implementation Notes

We're using `Distributions.jl` cause it's a nice, simple probability package. 

In [12]:
using Distributions
using Plots

INFO: Precompiling module Reexport.
INFO: Precompiling module FixedSizeArrays.
INFO: Precompiling module RecipesBase.
INFO: Precompiling module PlotUtils.
INFO: Precompiling module PlotThemes.
INFO: Precompiling module Showoff.
INFO: Recompiling stale cache file /home/henripal/.julia/lib/v0.5/FileIO.ji for module FileIO.


## Some problems in high dimensions.

Allright, let's generate some data from normal distributions in high dimensions and see how well we can recover stuff

In [214]:
N_observations = 100
N_dimensions = 2
mean_1 = 0
mean_2 = 2/sqrt(N_dimensions)
x1 = rand(Normal(mean_1, 1), N_observations, N_dimensions)
x2 = rand(Normal(mean_2, 1), N_observations, N_dimensions);

In [210]:
scatter(x1[:, 1], x1[:, 2])
scatter!(x2[:, 1], x2[:, 2], markercolor = :red) 

Let's spin off a quick k-means using always the correct initialization, and see what happens.

In [211]:
function find_k_mean(data, init, k, max_err = .1)
    old_err = Inf
    new_err = 1000000
    n = size(data)[1]
    assgn = Vector{Int}(n)
    means = init
    newdist = 0
    
    distances = Vector{Float64}(n)
    
    while abs(new_err - old_err) > max_err
       old_err = new_err
       new_err = 0 
    
       for i in 1:n 
            point = data[i, :]
            dist = Inf
            for (index, mean) in enumerate(means)
                newdist = norm(point-mean)
                if newdist < dist 
                    assgn[i] = index
                    dist = newdist 
                end
            end
            new_err = new_err + newdist
        end
        
        for index=1:k
            means[index] = mean(data[assgn .== index, :], 1)[:]
        end
    end
    
    return assgn 
end


find_k_mean (generic function with 2 methods)

In [212]:
result = find_k_mean(vcat(x1, x2), [x1[1,:], x2[1,:]], 2);

In [213]:
sum(result .== vcat(ones(N_observations), ones(N_observations)*2 )) / N_observations / 2

0.325

In [184]:
n = 3
norm(zeros(n) - ones(n))/sqrt(n)

1.0

In [182]:
sqrt(2)

1.4142135623730951